In [2]:
import cv2
import numpy as np
from smoothing import gaussian_smoothing_kernel
from sharpening import sharpening_kernel
from convolution import apply_convolution
import matplotlib.pyplot as plt

In [3]:
lena = cv2.imread('./Lena.jpg', 1)

cv2.imshow('Original Image', lena)
cv2.waitKey(0)
cv2.destroyAllWindows()

## 2.a: Take an RGB image and apply convolution with each kernel, separately on each channel.


In [3]:
lena_b = lena.copy()
lena_b[:, :, 1] = 0
lena_b[:, :, 2] = 0
print(lena_b.shape)

lena_g = lena.copy()
lena_g[:, :, 0] = 0
lena_g[:, :, 2] = 0
print(lena_g.shape)

lena_r = lena.copy()
lena_r[:, :, 0] = 0
lena_r[:, :, 1] = 0
print(lena_r.shape)


(512, 512, 3)
(512, 512, 3)
(512, 512, 3)


In [4]:
cv2.imshow("Lena Blue Channel", lena_b)
cv2.imshow("Lena Green Channel", lena_g)
cv2.imshow("Lena Red Channel", lena_r)


cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
bc, gc, rc = cv2.split(lena)

print(bc.shape, gc.shape, rc.shape)

(512, 512) (512, 512) (512, 512)


- Gaussian Smoothing Kernel

In [6]:
smk = gaussian_smoothing_kernel(5, 1)

print(smk)

smooth_bc = apply_convolution(bc, smk)
smooth_gc = apply_convolution(gc, smk)
smooth_rc = apply_convolution(rc, smk)

[[0.00296902 0.01330621 0.02193823 0.01330621 0.00296902]
 [0.01330621 0.0596343  0.09832033 0.0596343  0.01330621]
 [0.02193823 0.09832033 0.16210282 0.09832033 0.02193823]
 [0.01330621 0.0596343  0.09832033 0.0596343  0.01330621]
 [0.00296902 0.01330621 0.02193823 0.01330621 0.00296902]]


In [7]:
border_img_bc = smooth_bc["border_img"]
convo_bc = smooth_bc["raw_result"]
norm_bc = smooth_bc["normalized"]
norm_cropped_bc = smooth_bc["final"]

border_img_gc = smooth_gc["border_img"]
convo_gc = smooth_gc["raw_result"]
norm_gc = smooth_gc["normalized"]
norm_cropped_gc = smooth_gc["final"]

border_img_rc = smooth_rc["border_img"]
convo_rc = smooth_rc["raw_result"]
norm_rc = smooth_rc["normalized"]
norm_cropped_rc = smooth_rc["final"]

intermediate = cv2.merge((norm_cropped_bc, norm_cropped_gc, norm_cropped_rc))
final = np.round(cv2.normalize(intermediate, None, 0, 255, cv2.NORM_MINMAX)).astype(np.uint8)

In [8]:
blue_only = np.zeros((norm_cropped_bc.shape[0], norm_cropped_bc.shape[1], 3), dtype=np.uint8)
blue_only[:, :, 0] = 255 
blue_only[:, :, 1] = 255-norm_cropped_bc
blue_only[:, :, 2] = 255-norm_cropped_bc

green_only = np.zeros((norm_cropped_bc.shape[0], norm_cropped_bc.shape[1], 3), dtype=np.uint8)
green_only[:, :, 0] = 255-norm_cropped_gc
green_only[:, :, 1] = 255
green_only[:, :, 2] = 255-norm_cropped_gc

red_only = np.zeros((norm_cropped_bc.shape[0], norm_cropped_bc.shape[1], 3), dtype=np.uint8)
red_only[:, :, 0] = 255-norm_cropped_rc
red_only[:, :, 1] = 255-norm_cropped_rc
red_only[:, :, 2] = 255

test = cv2.merge((norm_cropped_bc, norm_cropped_gc, norm_cropped_rc))

cv2.imshow("Blue Channel Convolution Only", blue_only)
cv2.imshow("Green Channel Convolution Only", green_only)
cv2.imshow("Red Channel Convolution Only", red_only)
cv2.imshow("Original", lena)
cv2.imshow("Gaussian Smooth", test)

cv2.waitKey(0)
cv2.destroyAllWindows()

- Gaussian Sharpening (LoG) Kernel

In [17]:
bc2, gc2, rc2 = cv2.split(lena)

In [18]:
sh_k = sharpening_kernel(7,1.3)
print(sh_k)

[[ 0.00234583  0.00677597  0.01132721  0.0129268   0.01132721  0.00677597
   0.00234583]
 [ 0.00677597  0.01428519  0.01216823  0.00626028  0.01216823  0.01428519
   0.00677597]
 [ 0.01132721  0.01216823 -0.02518025 -0.05837777 -0.02518025  0.01216823
   0.01132721]
 [ 0.0129268   0.00626028 -0.05837777 -0.11144914 -0.05837777  0.00626028
   0.0129268 ]
 [ 0.01132721  0.01216823 -0.02518025 -0.05837777 -0.02518025  0.01216823
   0.01132721]
 [ 0.00677597  0.01428519  0.01216823  0.00626028  0.01216823  0.01428519
   0.00677597]
 [ 0.00234583  0.00677597  0.01132721  0.0129268   0.01132721  0.00677597
   0.00234583]]


In [19]:
bc2_m_c = apply_convolution(bc2, sh_k)

raw1 = bc2_m_c["raw_result"]

# Create a blue outline on white background
blue_outline = np.ones((raw1.shape[0], raw1.shape[1], 3), dtype=np.uint8) * 255 
blue_outline[:, :, 0] = 255 
blue_outline[:, :, 1] = raw1      
blue_outline[:, :, 2] = raw1    

cv2.imshow("Blue Channel Convolution", raw1)
cv2.imshow("Blue Outline on White", blue_outline)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [20]:
gc2_m_c = apply_convolution(gc2, sh_k)

raw2 = gc2_m_c["raw_result"]

# Create a blue outline on white background
freen_outline = np.zeros((raw2.shape[0], raw2.shape[1], 3), dtype=np.uint8)
freen_outline[:, :, 0] = raw2
freen_outline[:, :, 1] = 255  
freen_outline[:, :, 2] = raw2

cv2.imshow("Green Channel Convolution", raw2)
cv2.imshow("Green Outline on White", freen_outline)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [21]:
rc2_m_c = apply_convolution(rc2, sh_k)

raw3 = rc2_m_c["raw_result"]

# Create a blue outline on white background
red_outline = np.zeros((raw3.shape[0], raw3.shape[1], 3), dtype=np.uint8)
red_outline[:, :, 0] = raw3
red_outline[:, :, 1] = raw3
red_outline[:, :, 2] = 255

cv2.imshow("Red Channel Convolution", raw3)
cv2.imshow("Red Outline on White", red_outline)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [22]:
bgr_merge = cv2.merge((raw1, raw2, raw3))

cv2.imshow("Merged Outline Image", bgr_merge)
cv2.waitKey(0)
cv2.destroyAllWindows()

## 2.b : Convert the RGB image to HSV mode and apply convolution with each kernel, separately on each channel of HSV space.

In [4]:
lena_hsv = cv2.cvtColor(lena, cv2.COLOR_BGR2HSV)

cv2.imshow("Lena HSV", lena_hsv)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
h, s, v = cv2.split(lena_hsv)

print(h.shape)
print(s.shape)
print(v.shape)

cv2.imshow("Lena H Channel", h)
cv2.imshow("Lena S Channel", s)
cv2.imshow("Lena V Channel", v)

cv2.waitKey(0)
cv2.destroyAllWindows()

(512, 512)
(512, 512)
(512, 512)


- Gaussian Smoothing Kernel

In [6]:
sm_kernel = gaussian_smoothing_kernel(5, 1)

print(sm_kernel)

h_sm_convo = apply_convolution(h, sm_kernel)
s_sm_convo = apply_convolution(s, sm_kernel)
v_sm_convo = apply_convolution(v, sm_kernel)

[[0.00296902 0.01330621 0.02193823 0.01330621 0.00296902]
 [0.01330621 0.0596343  0.09832033 0.0596343  0.01330621]
 [0.02193823 0.09832033 0.16210282 0.09832033 0.02193823]
 [0.01330621 0.0596343  0.09832033 0.0596343  0.01330621]
 [0.00296902 0.01330621 0.02193823 0.01330621 0.00296902]]


In [ ]:
h_border = h_sm_convo["border_img"]
h_raw = h_sm_convo["raw_result"]
h_norm = h_sm_convo["normalized"]
h_final = h_sm_convo["final"]

s_border = s_sm_convo["border_img"]
s_raw = s_sm_convo["raw_result"]
s_norm = s_sm_convo["normalized"]
s_final = s_sm_convo["final"]

v_border = v_sm_convo["border_img"]
v_raw = v_sm_convo["raw_result"]
v_norm = v_sm_convo["normalized"]
v_final = v_sm_convo["final"]

convo_merge=cv2.merge([h_raw, s_raw, v_raw])
merge_norm = cv2.normalize(convo_merge, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
hsvtobgr = cv2.cvtColor(merge_norm, cv2.COLOR_HSV2BGR)

In [23]:
cv2.imshow("HSV Norm", merge_norm)
cv2.imshow("HSV to BGR", hsvtobgr)

cv2.waitKey(0)
cv2.destroyAllWindows()

- Sharpening Kernel

In [24]:
sh_kernel = sharpening_kernel(7, 1)

print(sh_kernel)

sh_h_convo = apply_convolution(h, sh_kernel)
sh_s_convo = apply_convolution(s, sh_kernel)
sh_v_convo = apply_convolution(v, sh_kernel)

[[ 3.14260486e-04  2.63207757e-03  8.57902057e-03  1.23763620e-02
   8.57902057e-03  2.63207757e-03  3.14260486e-04]
 [ 2.63207757e-03  1.74901468e-02  3.91926999e-02  4.30785586e-02
   3.91926999e-02  1.74901468e-02  2.63207757e-03]
 [ 8.57902057e-03  3.91926999e-02 -0.00000000e+00 -9.65323526e-02
  -0.00000000e+00  3.91926999e-02  8.57902057e-03]
 [ 1.23763620e-02  4.30785586e-02 -9.65323526e-02 -3.18309886e-01
  -9.65323526e-02  4.30785586e-02  1.23763620e-02]
 [ 8.57902057e-03  3.91926999e-02 -0.00000000e+00 -9.65323526e-02
  -0.00000000e+00  3.91926999e-02  8.57902057e-03]
 [ 2.63207757e-03  1.74901468e-02  3.91926999e-02  4.30785586e-02
   3.91926999e-02  1.74901468e-02  2.63207757e-03]
 [ 3.14260486e-04  2.63207757e-03  8.57902057e-03  1.23763620e-02
   8.57902057e-03  2.63207757e-03  3.14260486e-04]]


In [25]:
h_border_sh = sh_h_convo["border_img"]
h_raw_sh = sh_h_convo["raw_result"]
h_norm_sh = sh_h_convo["normalized"]
h_final_sh = sh_h_convo["final"]

s_border_sh = sh_s_convo["border_img"]
s_raw_sh = sh_s_convo["raw_result"]
s_norm_sh = sh_s_convo["normalized"]
s_final_sh = sh_s_convo["final"]

v_border_sh = sh_v_convo["border_img"]
v_raw_sh = sh_v_convo["raw_result"]
v_norm_sh = sh_v_convo["normalized"]
v_final_sh = sh_v_convo["final"]

hsv_sh_merge = cv2.merge([h_raw_sh, s_raw_sh, v_raw_sh])
hsv_sh_merge_norm = cv2.normalize(hsv_sh_merge, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
hsv_sh_back = cv2.cvtColor(hsv_sh_merge_norm, cv2.COLOR_HSV2BGR)


In [26]:
cv2.imshow("Original HSV", lena_hsv)
cv2.imshow("Norm Sharp", hsv_sh_merge_norm)
cv2.imshow("Final HSVtoBGR Sharp", hsv_sh_back)

cv2.waitKey(0)
cv2.destroyAllWindows()